In [8]:
import os, pickle

In [9]:
with open("../../../training_data/7.Extra_set/features.pkl", "rb") as f:
    extras_featuresd = pickle.load(f)

len(extras_featuresd), extras_featuresd

(9,
 {'7gqu':     Residues                                                          \
           pdb label_entity_id label_asym_id label_seq_id auth_asym_id   
  0       7gqu               1             A           12            A   
  1       7gqu               1             A           13            A   
  2       7gqu               1             A           14            A   
  3       7gqu               1             A           15            A   
  4       7gqu               1             A           16            A   
  ..       ...             ...           ...          ...          ...   
  414     7gqu               1             A          426            A   
  415     7gqu               1             A          427            A   
  416     7gqu               1             A          428            A   
  417     7gqu               1             A          429            A   
  418     7gqu               1             A          430            A   
  
                       

# Make predictions

## Gather data/features

In [5]:
os.system(f"ln -s {os.getcwd().rsplit('/', 3)[0]}/training_data/utils/external/fpocket/bin/fpocket fpocket") # os.getcwd().rsplit('/', 3)[0] --> absolute path of allodb_new

ln: failed to create symbolic link 'fpocket': File exists


256

In [6]:
os.system(f"ln -s {os.getcwd().rsplit('/', 3)[0]}/training_data/utils/external/mkdssp-4.4.0-linux-x64 mkdssp-4.4.0-linux-x64") # os.getcwd().rsplit('/', 3)[0] --> absolute path of allodb_new

ln: failed to create symbolic link 'mkdssp-4.4.0-linux-x64': File exists


256

In [13]:
os.system(f"ln -s /data/fnerin/nr_database nr") # os.getcwd().rsplit('/', 3)[0] --> absolute path of allodb_new

0

In [3]:
import subprocess, pymol2

In [14]:
for pdb in extras_featuresd:
    pdb = pdb.upper()
    os.makedirs(pdb, exist_ok=True)

    # os.system(f"ln -s {os.getcwd().rsplit('/', 1)[0]}/structures/{pdb.lower()}.pdb {pdb}/{pdb}.pdb") # pdb file needs a HEADER first line for mkdssp to work
    pdbf = f"{pdb}/{pdb}.pdb"
    if not os.path.isfile(pdbf):
        with (
            open(f"{os.getcwd().rsplit('/', 1)[0]}/structures/{pdb.lower()}.pdb", "r") as orig_pdbf,
            open(pdbf, "w") as f
        ):
            f.write(f"HEADER {pdb}\n")
            f.write(orig_pdbf.read())
    
    # os.system(f"./mkdssp-4.4.0-linux-x64 --output-format dssp {pdb}/{pdb}.pdb") # need to capture output
    dsspf = pdbf.replace(".pdb", ".dssp")
    if not os.path.isfile(dsspf):
        with open(dsspf, "w") as f:
            f.write(
                subprocess.run(["./mkdssp-4.4.0-linux-x64", "--output-format=dssp", pdbf], capture_output=True)
                .stdout.decode()
            )

    asnf = pdbf.replace(".pdb", ".asn")
    if not os.path.isfile(asnf):
        with pymol2.PyMOL() as pymol, open(pdbf.replace(".pdb", ".fasta"), "w") as f:#tempfile.NamedTemporaryFile("w+", suffix=".fasta") as f:
            pymol.cmd.load(pdbf, "prot")
            f.write(pymol.cmd.get_fastastr('prot'))
        
        os.system(f"cd nr && psiblast -query {os.getcwd()}/{pdbf.replace('.pdb', '.fasta')} -db nr -out {os.getcwd()}/{pdbf.replace('.pdb', '.out')} -num_iterations 3 -evalue 0.001 -outfmt 11 -out_pssm {os.getcwd()}/{asnf} -save_pssm_after_last_round -num_threads 5")

## Predict

Edited utils.py:
- to adjust the fpocket callable to '../fpocket'
- pocket filenames to be 1-indexed instead of 0-indexed `in_file = open(self.path + self.pdbid + '_out/pockets/pocket' + str(pockidx+1) + '_atm.pdb')`
- np.mean of neighboring PSSM etc... adjusted so that np.mean() uses axis=0 and does column-average
- function sequenceCode because in the .asn from local psiblast the sequence is hexadecimal-encoded and it needs to be decoded for the rest to work

Edited AllosESmain.py to adjust the relative location of `models.m` 

In [3]:
for pdb, feats in extras_featuresd.items():
    pdb = pdb.upper()
    chain = feats[('Residues', 'auth_asym_id')].unique().item()
    if os.path.isfile(f"{pdb}/{pdb}.asn") and not os.path.isfile(f"{pdb}/{pdb}_{chain}_result.csv"):
        os.system(f"cd {pdb} && python ../AllosES/AllosES/AllosESmain.py --PDBID {pdb} --CHAIN {chain}")

***** POCKET HUNTING BEGINS ***** 


mkdir: cannot create directory ‘./8F4S_out/pockets’: File exists


***** POCKET HUNTING ENDS ***** 
******Detection pockets...******
Read successfully, PSSM matrix being extracted...
Extraction successful, exporting file now...
********************************************
******Program starts running...******
******Start extracting features...******
******Feature integration completed!******
******Start prediction...******
******Prediction completed!******
→The Pocket Rankings:
  Prediction Pocket1:	6954:A,6957:A,6958:A,6953:A,6950:A,6949:A,6913:A,6917:A,6916:A,6868:A,6896:A,6897:A,6898:A,6929:A,6946:A,6947:A,6951:A,6911:A,6912:A,6866:A,6931:A,6914:A,6918:A,6909:A,6869:A,7088:A
  Prediction Pocket2:	6835:A,6838:A,7042:A,6843:A,6846:A,6842:A,7045:A,6885:A
  Prediction Pocket3:	6812:A,6816:A,6813:A,7028:A
  Prediction Pocket4:	6804:A,6805:A,6851:A,6854:A,6806:A,6808:A
  Prediction Pocket5:	6889:A,6884:A,6890:A,6887:A,6893:A,6891:A,6888:A
  Prediction Pocket6:	7083:A,7057:A,6861:A,7055:A,7056:A,7054:A
  Prediction Pocket7:	6819:A,6818:A,6821:A,7028:A,702

mkdir: cannot create directory ‘./8V81_out/pockets’: File exists


***** POCKET HUNTING ENDS ***** 
******Detection pockets...******
Read successfully, PSSM matrix being extracted...
Extraction successful, exporting file now...
********************************************
******Program starts running...******
******Start extracting features...******
******Feature integration completed!******
******Start prediction...******
******Prediction completed!******
→The Pocket Rankings:
  Prediction Pocket1:	496:A,495:A,1060:A,553:A,497:A,1260:A,270:A,961:A,267:A,546:A,1292:A,972:A,266:A,967:A,968:A,964:A,969:A,1290:A,1255:A,1370:A,1258:A,274:A,1254:A,1344:A,401:A,173:A,577:A,550:A,1291:A,965:A,544:A,548:A,549:A,494:A,1345:A,1337:A,465:A,1347:A,170:A,1348:A,493:A,1349:A,1351:A,1346:A,1288:A,1219:A,962:A,543:A,1371:A,1368:A,1289:A,402:A,1332:A,1330:A,437:A,621:A,439:A,438:A,440:A,442:A,464:A,572:A,269:A,499:A,541:A,542:A,1329:A,1331:A,1251:A,268:A,540:A,966:A,1053:A,547:A,1261:A,461:A,463:A,177:A,1056:A,175:A,172:A,1059:A,469:A,1250:A,1400:A,271:A,1375:A,460:A,

mkdir: cannot create directory ‘./9DNM_out/pockets’: File exists


***** POCKET HUNTING ENDS ***** 
******Detection pockets...******
Read successfully, PSSM matrix being extracted...
Extraction successful, exporting file now...
********************************************
******Program starts running...******
******Start extracting features...******
******Feature integration completed!******
******Start prediction...******
******Prediction completed!******
→The Pocket Rankings:
  Prediction Pocket1:	415:A,345:A,344:A,414:A,409:A,31:A,407:A,411:A,121:A,416:A,412:A,122:A,79:A,406:A,123:A,78:A,77:A,417:A,61:A,343:A,405:A,419:A,172:A,408:A,80:A,120:A,29:A
  Prediction Pocket2:	314:A,316:A,305:A,191:A,372:A,374:A,184:A,306:A,181:A,310:A,313:A,274:A,278:A,318:A,275:A,180:A,187:A,188:A,271:A,186:A,209:A,270:A,190:A,309:A,177:A,183:A,244:A,277:A,282:A
  Prediction Pocket3:	320:A,299:A,301:A,275:A,279:A,370:A,286:A,284:A,278:A,303:A,318:A,319:A
  Prediction Pocket4:	222:A,176:A,308:A,175:A,225:A,177:A,180:A,223:A,307:A,220:A,178:A,451:A,281:A,277:A,221:A,179:A

# Process

In [4]:
import pandas as pd

In [5]:
results = {}

for pdb, feats in extras_featuresd.items():
    pdb = pdb.upper()
    chain = feats[('Residues', 'auth_asym_id')].unique().item()
    csv = f"{pdb}/{pdb}_{chain}_result.csv"
    if os.path.isfile(csv):
        results[pdb.lower()] = {
            f"pocket{pocket.iloc[0]+1}": {
                "pro_ave": pocket["pro_ave"],
                "residues": pd.DataFrame(
                    (
                        res.split(":") 
                        for res in pocket["residues"].split(",")
                    ),
                    columns=["auth_seq_id", "auth_asym_id"]
                )
            }
            for i, pocket in pd.read_csv(csv).iterrows()
        }

len(results), results

(9,
 {'7gqu': {'pocket7': {'pro_ave': 0.6077189166132947,
    'residues':    auth_seq_id auth_asym_id
    0          601            A
    1          829            A
    2          573            A
    3          577            A
    4          578            A
    ..         ...          ...
    73         923            A
    74         551            A
    75         547            A
    76         883            A
    77         801            A
    
    [78 rows x 2 columns]},
   'pocket20': {'pro_ave': 0.4671694797381203,
    'residues':    auth_seq_id auth_asym_id
    0          916            A
    1          917            A
    2          920            A
    3          727            A
    4          919            A
    5          555            A
    6          552            A
    7          551            A
    8          846            A
    9          845            A
    10         849            A
    11         913            A
    12         706            A
    13

In [6]:
resultsf = "alloses_results.pkl"

with open(resultsf, "wb") as f:
    pickle.dump(results, f)